# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [181]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [182]:
csv_path = "output_data/cities.csv"
clean_cities_df = pd.read_csv(csv_path, encoding="utf-8")
clean_cities_df.head()

,city,country,lat,lng,temp,humidity,cloudiness,wind_speed
0,Solnechnyy,RU,50.72,136.63,59.43,54,100,3.24
1,Port Alfred,ZA,-33.59,26.89,49.82,63,16,21.61
2,Ixtapa,MX,20.70,-105.20,83.08,74,90,4.70
3,Ushuaia,AR,-54.80,-68.30,32.00,93,90,13.87
4,Fort Dodge,US,42.50,-94.17,65.35,60,1,3.36


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [183]:
# Store latitude and longitude in locations
locations = clean_cities_df[["lat", "lng"]]

# Fill NaN values and convert to float
rating = clean_cities_df["humidity"].astype(float)

In [184]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=rating, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [185]:
len(clean_cities_df)

485

In [186]:
#Drop any rows will null values
clean_cities_df = clean_cities_df.dropna(how='any')

In [187]:
len(clean_cities_df)

480

In [188]:
clean_cities_df["temp"] = clean_cities_df["temp"].astype(int)
clean_cities_df["humidity"] = clean_cities_df["humidity"].astype(int)
clean_cities_df["cloudiness"] = clean_cities_df["cloudiness"].astype(int)
clean_cities_df["wind_speed"] = clean_cities_df["wind_speed"].astype(float)

clean_cities_df.dtypes

city           object
country        object
lat           float64
lng           float64
temp            int64
humidity        int64
cloudiness      int64
wind_speed    float64
dtype: object

In [189]:
best_cities_df = clean_cities_df.loc[clean_cities_df["cloudiness"] == 0]
best_cities_df= best_cities_df.loc[best_cities_df["wind_speed"] <10]
best_cities_df= best_cities_df.loc[best_cities_df["temp"] < 85]
best_cities_df= best_cities_df.loc[best_cities_df["temp"] > 75]
best_cities_df= best_cities_df.loc[best_cities_df["humidity"] < 65]
len(best_cities_df)

9

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [190]:
#Store into variable named hotel_df
hotel_df=best_cities_df

In [191]:
#Add a "Hotel Name" column to the DataFrame
hotel_df["hotel_name"] = ""
hotel_df.head()

,city,country,lat,lng,temp,humidity,cloudiness,wind_speed,hotel_name
22,Nola,IT,40.92,14.53,77,64,0,1.12,
76,Iranshahr,IR,27.20,60.68,82,15,0,6.02,
130,Chengzhong,CN,30.95,113.55,84,62,0,6.71,
161,Buraidah,SA,26.33,43.98,80,32,0,4.70,
193,Wewak,PG,-3.55,143.63,84,62,0,5.99,


In [192]:
# params dictionary to update each iteration
params = {
    "radius": 50000,
    "types": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["lat"]
    lng = row["lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    response = requests.get(base_url, params=params)

    # convert to json
    find_hotel = response.json()

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    hotel_df.loc[index, "hotel_name"] = find_hotel["results"][0]["name"]
    hotel_df   

In [219]:
hotel_df

,city,country,lat,lng,temp,humidity,cloudiness,wind_speed,hotel_name
22,Nola,IT,40.92,14.53,77,64,0,1.12,Naples
76,Iranshahr,IR,27.20,60.68,82,15,0,6.02,Iranshahr
130,Chengzhong,CN,30.95,113.55,84,62,0,6.71,Wuhan
161,Buraidah,SA,26.33,43.98,80,32,0,4.70,Buraydah
193,Wewak,PG,-3.55,143.63,84,62,0,5.99,Wewak
250,Alice Springs,AU,-23.70,133.88,76,11,0,3.36,Alice Springs
285,Aswān,EG,24.09,32.91,81,24,0,7.07,Aswan
345,Sakakah,SA,29.97,40.21,84,13,0,8.05,Sakaka
446,Jalu,LY,29.03,21.55,78,46,0,5.48,Awjilah


In [229]:
# Add marker layer ontop of heat map
# I have spent a lot of time trying to figure out how to label these markers, 
#but I just don't know how, and can't figure out any of the online help.

marker_locations = hotel_df[['lat', 'lng']]

markers = gmaps.marker_layer(marker_locations,
    info_box_content=[f"{city}, {country}" for row in hotel_df ])
fig.add_layer(markers)
fig


Figure(layout=FigureLayout(height='420px'))